**TASK 1**

In [97]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn import metrics
import random
import seaborn as sns
from scipy import stats
from copy import deepcopy
from sklearn.cluster import KMeans
from datetime import datetime

In [3]:
from google.colab import files
data = files.upload()

Saving football.csv to football.csv


In [98]:
data = pd.read_csv('football.csv')
data.head()

,Team,winsInSeason2016,winsInSeason2017
0,X1,3,5
1,X2,3,4
2,X3,2,8
3,X4,2,3
4,X5,6,2


In [99]:
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset

In [100]:
def lineToTuple(line):
    # remove leading/trailing witespace and newlines
    cleanLine = line.strip()
    # get rid of quotes
    cleanLine = cleanLine.replace('"', '')
    # separate the fields
    lineList = cleanLine.split(",")
    # convert strings into numbers
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    return lineTuple

In [102]:
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

In [103]:
def isValidNumberString(s):
    if len(s) == 0:
        return False
    if  len(s) > 1 and s[0] == "-":
        s = s[1:]
    for c in s:
        if c not in "0123456789.":
            return False
    return True

In [104]:
def distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(0, len(instance1)-1):
        sumOfSquares += (instance1[i+1] - instance2[i])**2
    return sumOfSquares


In [105]:
def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes)
    for instance in instanceList:
        for i in range(0, numAttributes):
            means[i+1] += instance[i]
   
    for i in range(0, numAttributes):
        means[i+1] /= float(numInstances)
    return tuple(means)

In [106]:
def assign(instance, centroids, dist):
    minDistance = dist(centroids[0],instance)
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = dist(centroids[i], instance )
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

In [107]:
def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

In [108]:
def assignAll(instances, centroids, labels, dist):
    clusters = createEmptyListOfLists(len(centroids))
    classclusters = createEmptyListOfLists(len(centroids))
    i=0
    for instance in instances:
        clusterIndex = assign(instance, centroids, dist)
        clusters[clusterIndex].append(instance)
        classclusters[clusterIndex].append(labels[i])
        i=i+1
    return clusters, classclusters

In [138]:
def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "CENTROID" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

In [143]:
def kmeans(instances, k, dist,labels, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
#         canvas = prepareWindow(instances)
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        #paintClusters2D(canvas, clusters, centroids, "Initial centroids")
#         time.sleep(delay)
    iteration = 0
    while (centroids != prevCentroids and iteration <1):
        iteration += 1
        clusters, votelabels = assignAll(instances, centroids, labels, dist)
        #if animation:
            #paintClusters2D(canvas, clusters, centroids, "Assign %d" % iteration)
            #time.sleep(delay)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, dist)
#         if animation:
#             paintClusters2D(canvas, clusters, centroids,
#                             "Update %d, withinss %.1f" % (iteration, withinss))
#             time.sleep(delay)
    print('ITERATIONS: ', iteration)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = votelabels
    return result

In [111]:
def computeWithinss(clusters, centroids, dist):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += dist(centroid, instance)
    return result


In [112]:
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i )
        trialClustering = kmeans(instances, k)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering


In [127]:
def euclidean(centroid, data):
    sum1=[]
    if len(centroid) == len(data)+1:
        centroid=centroid[1:]
    for i in range (0, len(data)):
        sum1.append((centroid[i- data[i])**2)
    
    
    euclidean= math.sqrt(sum(sum1))
    return euclidean

In [150]:
def manhatten(x,y):
    res=[]
    if len(x) == len(y)+1:
        x=x[1:]
    for i in range(len(x)):
        res.append(abs(x[i]-y[i]))
        
    return sum(res) 

In [149]:
def sse(clustering, centroids):
    i=0
    SSE=0
    for clust in clustering:
        for data in clust:
            SSE=SSE+distance(centroids[i], data)
        i=i+1
    return SSE

In [155]:
labels =data.Team

In [157]:
data = data.drop(['Team'], axis=1)

In [158]:
data

,winsInSeason2016,winsInSeason2017
0,3,5
1,3,4
2,2,8
3,2,3
4,6,2
5,6,4
6,7,3
7,7,4
8,8,5
9,7,6


In [173]:
a = np.array([[4, 6], [5, 4]])

In [161]:
dataset = data.values.tolist()

In [174]:
x = data.winsInSeason2016
y = data.winsInSeason2017

**ANSWER : 1**

In [175]:
clustering = kmeans(dataset, 2, manhatten,labels, True, initCentroids=[[4,6],[5,4]])
printTable(clustering["centroids"])
print(clustering['clusters'])
print('SSE: ',sse(clustering['clusters'],clustering['centroids']))

ITERATIONS:  1
CENTROID0	4.00 6.33 
CENTROID1	5.57 3.57 
[[[3, 5], [2, 8], [7, 6]], [[3, 4], [2, 3], [6, 2], [6, 4], [7, 3], [7, 4], [8, 5]]]
SSE:  54.09523809523809


**ANSWER : 2**

In [176]:
clustering = kmeans(dataset_f, 2, euclidean,labels, True, initCentroids=[[4,6],[5,4]])
printTable(clustering["centroids"])
print(clustering['clusters'])
print('SSE: ',sse(clustering['clusters'],clustering['centroids']))

ITERATIONS:  1
CENTROID0	2.50 6.50 
CENTROID1	5.75 3.88 
[[[3, 5], [2, 8]], [[3, 4], [2, 3], [6, 2], [6, 4], [7, 3], [7, 4], [8, 5], [7, 6]]]
SSE:  47.375


**ANSWER : 3**

In [171]:
clustering = kmeans(dataset_f, 2, manhatten,labels, True, initCentroids=[[3,3],[8,3]])
printTable(clustering["centroids"])
print(clustering['clusters'])
print('SSE: ',sse(clustering['clusters'],clustering['centroids']))

ITERATIONS:  1
CENTROID0	2.50 5.00 
CENTROID1	6.83 4.00 
[[[3, 5], [3, 4], [2, 8], [2, 3]], [[6, 2], [6, 4], [7, 3], [7, 4], [8, 5], [7, 6]]]
SSE:  27.833333333333332


**ANSWER 4:**

In [172]:
clustering = kmeans(dataset_f, 2, manhatten,labels, True, initCentroids=[[3,2],[4,8]])
printTable(clustering["centroids"])
print(clustering['clusters'])
print('SSE: ',sse(clustering['clusters'],clustering['centroids']))

ITERATIONS:  1
CENTROID0	4.86 3.57 
CENTROID1	5.67 6.33 
[[[3, 5], [3, 4], [2, 3], [6, 2], [6, 4], [7, 3], [7, 4]], [[2, 8], [8, 5], [7, 6]]]
SSE:  57.904761904761905
